In [ ]:
import numpy as np
%matplotlib ipympl
from matplotlib import pyplot as plt
import scipy.signal as signal
from scipy.io import wavfile
import IPython
import pretty_midi
from pqdm.processes import pqdm
import warnings
import sounddevice as sd

In [ ]:
samplerate = 44.1e3

In [ ]:
def release_window(t, k = 0.01):
    value = np.sqrt(1 - 1/k * t**2)
    return np.where(value > 0, value, 0)

plt.figure()
plt.plot(np.linspace(0, 1, 10000), release_window(np.linspace(0, 1, 10000)));
plt.ylim(-0.1, 1.1)

In [ ]:
def falloff_window(t):
    return (1 - 1/(1 + np.exp(-(1 * t - 1)))) / (1 - 1/(1 + np.exp(-(1 * 0 - 1))))

plt.figure()
plt.plot(np.linspace(-10, 10, 10000), falloff_window(np.linspace(-10, 10, 10000)));
plt.xlim(-0.01, 3)
plt.ylim(-0.1, 1.1)

In [ ]:
piano_window_defaults = (40, 2 * np.pi * 30)

def piano_response(t, b, wn):
    return np.exp(-b * t) * np.sin(wn * t + 1.5 * np.pi) + 1

plt.figure()
plt.plot(np.linspace(0, 1, 400000), piano_response(np.linspace(0, 1, 400000), *piano_window_defaults));

In [ ]:
timbre_defaults=[1.0,.3,.8,.2,.4,.2,.2,.1,.1]

def midi_to_freq(code):
    return 2**((code-69)/12.0) * 440

def make_note(f0, coeffs = timbre_defaults, ditter = 0.1):
    return f0 * (np.arange(len(coeffs)) + 1)

print(make_note(midi_to_freq(69)))

In [ ]:
class Nota:
    
    def __init__(self, freqs, coeffs = timbre_defaults, curr_phases = None, curr_time = 0, curr_release_time = 0, released = False) -> None:
        self.freqs = freqs
        self.coeffs = coeffs
        
        if curr_phases is not None:
            self.curr_phases = curr_phases
        else:
            self.curr_phases = np.zeros(len(freqs))
            
        self.curr_time = curr_time
        self.curr_release_time = curr_release_time
        self.released = released
        
        pass

In [ ]:
def make_buffer(notes, samplerate = samplerate, buffer_size = 1024, ditter = 0.01):
    dt = 1 / samplerate
    ts = np.linspace(0, buffer_size * dt, buffer_size + 1)
    buffer = np.zeros(buffer_size + 1)
    
    new_notes = {}
    
    for freq, nota in notes.items():
                
        curr_phases = np.zeros(len(nota.curr_phases))
        
        for i in range(len(nota.freqs)):
        
            ps = nota.curr_phases[i] + 2 * np.pi * nota.freqs[i] * ts
            ps = np.random.uniform(1 - ditter / 100, 1 + ditter / 100, len(ps)) * ps
        
            if nota.released:
                buffer += nota.coeffs[i] * np.sin(ps) * piano_response(ts + nota.curr_time, *piano_window_defaults) \
                                                      * falloff_window(ts + nota.curr_time) \
                                                      * release_window(ts + nota.curr_release_time)
            else:
                buffer += nota.coeffs[i] * np.sin(ps) * piano_response(ts + nota.curr_time, *piano_window_defaults) \
                                                      * falloff_window(ts + nota.curr_time)
                             
            curr_phases[i] = ps[-1] % (2 * np.pi)
            
        curr_time = (buffer_size + 1) * dt + nota.curr_time
        if nota.released:
            curr_release_time = (buffer_size + 1) * dt + nota.curr_release_time
        
            if release_window(curr_release_time) > 0:
                new_notes[freq] = Nota(nota.freqs, nota.coeffs, curr_phases, curr_time, curr_release_time, nota.released)
        else:
            new_notes[freq] = Nota(nota.freqs, nota.coeffs, curr_phases, curr_time, nota.curr_release_time, nota.released)
        
    return (buffer[:-1], new_notes)

In [ ]:
def simulate_instrument(sheet, tmax = 10, samplerate = samplerate, buffer_size = 1024, ditter = 0.01):
    sound = np.zeros(0)
    t = 0
    
    notes = {}
    
    while t < tmax:
        for note in sheet:
            if t >= note[1][1] and t <= note[1][1] + (buffer_size - 1) / samplerate and note[0] in notes:
                notes[note[0]].released = True
            if t >= note[1][0] and t <= note[1][0] + (buffer_size - 1) / samplerate:
                notes.update({note[0] : Nota(make_note(note[0]))})

        (buffer, notes) = make_buffer(notes, samplerate, buffer_size, ditter)
        
        sound = np.append(sound, buffer)
        t += buffer_size / samplerate
               
    return sound

sheet = [
    (440, (0, 0.5)),
    (493.88, (0, 0.5)),
    (440, (0.5, 0.75)),
    (440, (1, 1.5)),
]

sound = simulate_instrument(sheet, 2)

plt.figure()
plt.plot(sound)

IPython.display.Audio(sound, rate=samplerate)

In [ ]:
buffer_size = 1024

note = [Nota(make_note(440))]

ts = np.linspace(0, (buffer_size - 1) / samplerate, buffer_size)

plt.figure()

X = 200

sound = np.zeros(0)

for i in range(X):
    buffer, note = make_buffer(note, buffer_size = buffer_size)
    sound = np.append(sound, buffer)
    plt.plot(ts + i * buffer_size / samplerate, buffer[:buffer_size], marker='.')
note[0].released = True
for i in range(i+1, X + 20):
    buffer, note = make_buffer(note, buffer_size = buffer_size)
    sound = np.append(sound, buffer)
    plt.plot(ts + i * buffer_size / samplerate, buffer[:buffer_size], marker='.')
    
plt.figure()
plt.plot(sound, marker="o")
#plt.plot(np.abs(signal.hilbert(sound)))
    
plt.figure()
plt.specgram(sound, Fs = samplerate, scale="linear");

IPython.display.Audio(sound, rate=samplerate)   
#sd.play(sound, samplerate)

In [ ]:
def make_chord(midi_codes, timbre=timbre_defaults, piano_parameters=piano_window_defaults, duration=0.5):
    t = np.linspace(0, duration, int(duration * samplerate))
    y = np.zeros_like(t)
    for code in midi_codes:
        if code is not None:
            f0 = midi_to_freq(code)
            note = make_note(f0, timbre, t)
            y = note * piano_window(np.size(note), samplerate, *piano_parameters)
    return t, y

t, p = make_chord([69])
IPython.display.Audio(p, rate=samplerate)

In [ ]:
midi_data = pretty_midi.PrettyMIDI("Just-Can't-Get-Enough.mid")
duration = midi_data.get_end_time()

def make_melody(instrument):
    sound = np.zeros(int(duration*samplerate+1))
    for note in instrument.notes:
        _, chord = make_chord([note.pitch], duration=note.end-note.start)
        sound[int(note.start*samplerate):int(note.start*samplerate)+len(chord)] += chord
    return sound

warnings.filterwarnings('ignore')
sounds = pqdm(midi_data.instruments, make_melody, n_jobs=len(midi_data.instruments))
sound = np.sum(sounds, axis=0)

In [ ]:
IPython.display.Audio(sound[:np.size(sound)//1], rate=samplerate)

In [ ]:
plt.figure()
plt.specgram(sound , Fs = samplerate, NFFT = 1000)
plt.ylim(0, 20000);

In [ ]:
sd.play(generated, samplerate)